# Cell 1: Importing Libraries

In [ ]:
# Import necessary libraries for data manipulation, image processing, and machine learning
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import PIL.Image
import os
from tqdm import tqdm
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# Cell 2: Mounting Google Drive

In [ ]:
from google.colab import drive
# Mount Google Drive to access the dataset stored there
drive.mount('/content/drive', force_remount=True)

# Cell 3: Data Preparation

In [ ]:
# Load dataset from CSV file
df = pd.read_csv("drive/MyDrive/Group Project - Art Contest/Training Data/art_contest_dataset.csv")
# Group dataset by file name
grouped = df.groupby('file_name')
unique_fn = grouped.groups.keys()

# Split the dataset into training and test sets
train_fns, test_fns = train_test_split(list(unique_fn), test_size=0.2, random_state=42)
train_data = df[df['file_name'].isin(train_fns)]
test_data = df[df['file_name'].isin(test_fns)]

# Cell 4: Saving Train and Test Sets

In [ ]:
# Save the split train and test data into CSV files
train_data.to_csv("drive/MyDrive/Group Project - Art Contest/Training Data/train.csv", index=False)
test_data.to_csv("drive/MyDrive/Group Project - Art Contest/Training Data/test.csv", index=False)

# Cell 5: Dataset Class Definition

In [ ]:
class CustomDataset(Dataset):
    """
    Custom dataset class for loading and transforming image data.
    """

    def __init__(self, data, transform=None):
        """
        Initialize the dataset with file paths and optional transformations.
        """
        self.data = pd.read_csv(data)
        self.transform = transform

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieve an image and its label at the specified index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: (image, label) where image is the transformed image tensor and label is a float tensor of the label.
        """
        # Construct image file path
        img_name = self.data.loc[idx, "file_name"].replace('/', '')
        img_name = 'Copy of ' + img_name
        img_path = os.path.join('/content/drive/MyDrive/Group Project - Art Contest/Training Data/all_training_art/', img_name)

        # Load and convert the image to RGB
        image = PIL.Image.open(img_path).convert("RGB")

        # Map category labels to numerical values
        label_map = {"good_art": 1, "bad_art": 0}
        label = torch.tensor(label_map[self.data.loc[idx, "category_id"]])

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label.float()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize images to fit model input size
    transforms.ToTensor()          # Convert images to PyTorch tensors
])

# Cell 6: Setting File Paths

In [ ]:
# Define file paths for training and test data
path = "drive/MyDrive/Group Project - Art Contest/Training Data/"
train = os.path.join(path, 'train.csv')
test = os.path.join(path, 'test.csv')

# Cell 7: Creating Datasets

In [ ]:
# Initialize custom datasets for training and testing
train_dataset = CustomDataset(train, transform=transform)
test_dataset = CustomDataset(test, transform=transform)

# Cell 8: DataLoader Initialization

In [ ]:
# Define batch size for data loading
batch_size = 32
# Initialize DataLoader for batching operations
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Cell 9: Setting Up Device

In [ ]:
# Check for GPU availability, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Cell 10: Model Preparation

In [ ]:
from torch.optim import Adam
# Load a pre-trained ResNet-50 model
model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

# Modify the model to fit the specific classification task
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 2)  # Output layer for binary classification
)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Set up the optimizer for training the fully connected layers
optimizer = Adam(model.fc.parameters(), lr=0.001)

# Cell 11: Freezing Layers

In [ ]:
# Freeze all layers except the fully connected layers
for name, param in model.named_parameters():
    param.requires_grad = False
    if name.startswith('fc'):
        param.requires_grad = True

# Cell 12: Validation Function

In [ ]:
def valid(dataloader, model):
    """
    Validate the model on a given dataloader.
    """
    model.eval()  # Set model to evaluation mode
    correct_predictions = 0
    total_samples = len(dataloader.dataset)

    with torch.no_grad():
        for images, labels in dataloader:
            labels = labels.type(torch.LongTensor)
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct_predictions += (outputs.argmax(1) == labels).type(torch.float).sum().item()

    accuracy = correct_predictions / total_samples * 100
    return accuracy

# Cell 13: Training Loop

In [ ]:
# Set number of epochs for training
num_epochs = 10
best_accuracy = 0
model.to(device)

for epoch in tqdm(range(num_epochs)):
    model.train()  # Set model to training mode
    total_loss = 0.0

    for images, labels in train_loader:
        labels = labels.type(torch.LongTensor)
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {average_loss:.4f}")

    accuracy = valid(test_loader, model)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, 'model.pth')
        print(f'Epoch {epoch + 1}: New best model with accuracy {best_accuracy:.2f}% saved.\n\n')

# Cell 14: Loading the Best Model

In [ ]:
# Load the best model from file
PATH = '/content/model.pth'
model = torch.load(PATH).to(device)

# Cell 15: Test Evaluation

In [ ]:
# Evaluate the model on the test dataset
model.eval()  # Set model to evaluation mode
correct_predictions = 0
total_samples = len(test_loader.dataset)
predicted = []
truth = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        truth.extend(labels.cpu().numpy())
        correct_predictions += (outputs.argmax(1) == labels).type(torch.float).sum().item()
        predicted.extend(outputs.argmax(1).cpu().numpy())

accuracy = correct_predictions / total_samples * 100
print(f"Test Accuracy: {accuracy:.2f}%")

# Cell 16: Confusion Matrix

In [ ]:
# Calculate and display the confusion matrix
cm = confusion_matrix(truth, predicted)
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)

# Cell 17: Displaying Confusion Matrix

In [ ]:
# Visualize the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

# Cell 18: Installing Additional Packages

In [ ]:
# Install multiple Python packages using pip in a single command
!pip install tiktoken cohere openai requests

# Cell 19: Setting OpenAI API Key

In [ ]:
# Set the OpenAI API key as an environment variable
%env OPENAI_API_KEY=<your_actual_key_here>

# Cell 20: Image Generation and Ranking

In [ ]:
from openai import OpenAI
from ipywidgets import interact, widgets, VBox, HBox, HTML, Image as IPImage
from IPython.display import display, Image, HTML
from PIL import Image as PILImage
from io import BytesIO
import requests
import threading
import time
import torch
from torchvision import transforms

# Define the classes for the art
class_names = ["good_art", "bad_art"]

# Initialize global variable to store the start time of image generation
start_time = None

# Initialize lists to store the generated images, prompts, and confidence scores
generated_images = []
generated_prompts = []
generated_images_with_confidence = []

def generate_ai_image(prompt, result_list, index, loading_bar):
    """
    Generate an AI image based on the prompt and update the result list and loading bar.

    Args:
    - prompt (str): The prompt to generate the image from.
    - result_list (list): The list to store generated images.
    - index (int): The index in the result list to store the generated image.
    - loading_bar (widgets.IntProgress): The loading bar widget to update progress.
    """
    client = OpenAI()
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url
    image_response = requests.get(image_url)
    image_bytes = image_response.content
    result_list[index] = image_bytes
    loading_bar.value += 1

def resize_images(images):
    """
    Resize the input images to the required size for the model.

    Args:
    - images (list): List of PIL.Image objects.

    Returns:
    - torch.Tensor: Tensor containing the stacked resized image tensors.
    """
    transformed_images = []
    for img in images:
        resized_img = img.resize((224, 224), PILImage.LANCZOS)
        tensor_img = transforms.ToTensor()(resized_img)
        transformed_images.append(tensor_img)
    return torch.stack(transformed_images)

def generate_and_display_images(prompt, model, class_names):
    """
    Generate and display images based on the given prompt using the specified model.

    Args:
    - prompt (str): The prompt to generate images for.
    - model (torch.nn.Module): The neural network model to rank images.
    - class_names (list): The list of class names for classification.
    """
    num_images = 5  # Set the desired number of images to generate
    result_list = [None] * num_images
    loading_bar = widgets.IntProgress(min=0, max=num_images, description='Generating:')
    display(loading_bar)

    global start_time
    start_time = time.time()  # Record the start time for the generation process

    # Start threads for image generation
    threads = [threading.Thread(target=generate_ai_image, args=(prompt, result_list, i, loading_bar)) for i in range(num_images)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()  # Wait for all threads to complete

    # Convert the bytes to PIL images and perform resizing
    pil_images = [PILImage.open(BytesIO(img)) for img in result_list if img is not None]
    resized_images = resize_images(pil_images)

    # Rank the images based on the neural network
    sorted_images, ranked_info = rank_images(resized_images, model, class_names)

    # Initialize lists for the image widgets and labels
    ranked_widgets_images = []
    confidence_labels = []
    class_name_labels = []

    # Create labels for the confidence and class names
    for _, class_name, confidence in ranked_info:
        confidence_labels.append(widgets.Label(f'Confidence: {confidence:.2%}'))
        class_name_labels.append(widgets.Label(f'Class: {class_name}'))

    # Create image widgets and associate them with confidence and class labels
    for i, img in enumerate(sorted_images):
        image_bytes = img.permute(1, 2, 0).cpu().numpy() * 255
        image_bytes = image_bytes.astype('uint8')
        image_pil = PILImage.fromarray(image_bytes)
        image_bytes_io = BytesIO()
        image_pil.save(image_bytes_io, format='PNG')
        image_bytes_io.seek(0)
        img_widget = widgets.Image(value=image_bytes_io.read(), format='png', width=224, height=224)
        hbox = HBox([img_widget, VBox([confidence_labels[i], class_name_labels[i]])])
        ranked_widgets_images.append(hbox)

    # Create a tab widget to display ranked images with labels
    tab = widgets.Tab()
    tab.children = ranked_widgets_images
    for i in range(len(ranked_widgets_images)):
        tab.set_title(i, str(i))

    # Display elapsed time and the tab widget
    elapsed_time = time.time() - start_time
    display(HTML(f'Total Elapsed Time: {elapsed_time:.2f} seconds'))
    display(tab)

def rank_images(images, model, class_names):
    """
    Rank images using the provided model and return sorted images and ranking information.

    Args:
    - images (torch.Tensor): The images to rank.
    - model (torch.nn.Module): The neural network model to use for ranking.
    - class_names (list): The list of class names corresponding to the model's outputs.

    Returns:
    - list: Sorted images based on the confidence scores.
    - list: Ranking information including image tensor, predicted class name, and confidence score.
    """
    ranked_images = []
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for image in images:
            # Process each image through the model and collect ranking info
            image = image.unsqueeze(0).to(device)
            outputs = model(image)
            probabilities = torch.softmax(outputs, dim=1)[0]
            predicted_class_index = torch.argmax(probabilities).item()
            confidence_score = probabilities[predicted_class_index].item()
            predicted_class_name = class_names[predicted_class_index]
            ranked_images.append((image.squeeze(0), predicted_class_name, confidence_score))
    ranked_images.sort(key=lambda x: x[2], reverse=True)
    return [img for img, _, _ in ranked_images], ranked_images

# Widget setup for user interaction
genre_dropdown = widgets.Dropdown(
    options=["Abstract Expressionism", "Abstract", "Baroque", "Cubism", "Impressionism", "Minimalism", "Pop Art", "Realism", "Realism", "Renaissance", "Romanticism", "Surrealism"],
    description="Genre:"
)
prompt_input = widgets.Text(description="Prompt:", value="a portrait of a man")
generate_button = widgets.Button(description="Generate Images")

def on_generate_button_clicked(btn):
    """
    Handle the click event on the 'Generate Images' button.

    Args:
    - btn (widgets.Button): The button instance that was clicked.
    """
    prompt = prompt_input.value
    selected_genre = genre_dropdown.value
    generate_and_display_images(f"{prompt} in the style of {selected_genre} art", model, class_names)

generate_button.on_click(on_generate_button_clicked)

# Display the widgets
display(VBox([genre_dropdown, prompt_input, generate_button]))
